# Section: Federated Learning

# Lesson: Introducing Federated Learning

<font color=red>Federated Learning is a technique for training Deep Learning models on data to which you do not have access.</font> Basically:

<font color=blue>Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.</font>

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

W0803 17:56:12.782911 4627559872 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0803 17:56:13.163151 4627559872 deprecation_wrapper.py:119] From /Users/lek/miniconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

<font color=red>The essence of Federated Learning is the ability to train models in parallel on a wide number of machines.</font> Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, <font color=red>instead of using Torch tensors - we're now going to work with **pointers** to tensors.</font> Let me show you what I mean. <font color=blue>First, let's create a "pretend" machine owned by a "pretend" person</font> - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{34196116615: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #objects:1>

In [14]:
x.id_at_location

34196116615

In [15]:
x.id

9856897708

In [16]:
x.owner

<VirtualWorker id:me #objects:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:9856897708 -> bob:34196116615]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
# try this project here!

In [21]:
alice = sy.VirtualWorker(hook, id='alice')

In [31]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [32]:
x_ptr = x.send(bob, alice)

In [34]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:33811829268 -> bob:71615211823]
	-> (Wrapper)>[PointerTensor | me:21197454072 -> alice:84746910559]

In [35]:
x_ptr.child

MultiPointerTensor>{'bob': (Wrapper)>[PointerTensor | me:33811829268 -> bob:71615211823], 'alice': (Wrapper)>[PointerTensor | me:21197454072 -> alice:84746910559]}

In [36]:
x_ptr.child.child

{'bob': (Wrapper)>[PointerTensor | me:33811829268 -> bob:71615211823],
 'alice': (Wrapper)>[PointerTensor | me:21197454072 -> alice:84746910559]}

In [37]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [28]:
x = th.tensor([1,2,3,4,5]).send(bob,alice)

In [29]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

In [30]:
x

AttributeError: 'Tensor' object has no attribute 'child'

# Lesson: Introducing Remote Arithmetic

In [34]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [35]:
x

(Wrapper)>[PointerTensor | me:73784941666 -> bob:8448304359]

In [36]:
y

(Wrapper)>[PointerTensor | me:21299323032 -> bob:63942701325]

In [37]:
z = x + y

In [38]:
z

(Wrapper)>[PointerTensor | me:67523050441 -> bob:55873920706]

In [39]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [40]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:66095067947 -> bob:60841081525]

In [41]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [42]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [43]:
z = (x + y).sum()

In [44]:
z.backward()

(Wrapper)>[PointerTensor | me:43061343639 -> bob:78361453055]

In [45]:
x = x.get()

In [46]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [47]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
# try this project here!

In [35]:
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [36]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob) 

In [37]:
pred = input.mm(weigths)

In [38]:
loss = ((pred - target)**2).sum()

In [39]:
loss.backward()

(Wrapper)>[PointerTensor | me:87871516037 -> bob:26320082206]

In [42]:
weights.data.sub_(weights.grad * 0.1)
weights.grad *= 0

RuntimeError: Command "__mul__" is not a supported Torch operation.

In [43]:
for i in range(10):
    pred = input.mm(weights)
    
    loss = ((pred - target)**2).sum()
    
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0
    
    print(loss.get().data)
    

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [44]:
bob = bob.clear_objects()

In [45]:
bob._objects

{}

In [46]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [47]:
bob._objects

{8587764814: tensor([1, 2, 3, 4, 5])}

In [48]:
del x

In [49]:
bob._objects

{}

In [64]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [65]:
x.child.garbage_collect_data

True

In [66]:
bob._objects

{8702103425: tensor([1, 2, 3, 4, 5]),
 93236886462: tensor([1, 2, 3, 4, 5]),
 17917990552: tensor([1, 2, 3, 4, 5]),
 81519178095: tensor([1, 2, 3, 4, 5])}

In [67]:
x = "asdf"

In [68]:
bob._objects

{8702103425: tensor([1, 2, 3, 4, 5]),
 93236886462: tensor([1, 2, 3, 4, 5]),
 17917990552: tensor([1, 2, 3, 4, 5])}

In [69]:
Out

{12: (Wrapper)>[PointerTensor | me:17186469775 -> bob:98080682002],
 17: (Wrapper)>[PointerTensor | me:85530511808 -> bob:18675112323],
 29: (Wrapper)>[PointerTensor | me:50372017420 -> bob:13265151605],
 39: (Wrapper)>[PointerTensor | me:87871516037 -> bob:26320082206],
 45: {8702103425: tensor([1, 2, 3, 4, 5]),
  93236886462: tensor([1, 2, 3, 4, 5]),
  17917990552: tensor([1, 2, 3, 4, 5])},
 47: {8702103425: tensor([1, 2, 3, 4, 5]),
  93236886462: tensor([1, 2, 3, 4, 5]),
  17917990552: tensor([1, 2, 3, 4, 5])},
 49: {8702103425: tensor([1, 2, 3, 4, 5]),
  93236886462: tensor([1, 2, 3, 4, 5]),
  17917990552: tensor([1, 2, 3, 4, 5])},
 51: True,
 52: {8702103425: tensor([1, 2, 3, 4, 5]),
  93236886462: tensor([1, 2, 3, 4, 5]),
  17917990552: tensor([1, 2, 3, 4, 5])},
 54: {8702103425: tensor([1, 2, 3, 4, 5]),
  93236886462: tensor([1, 2, 3, 4, 5]),
  17917990552: tensor([1, 2, 3, 4, 5])},
 57: (Wrapper)>[PointerTensor | me:84964866159 -> bob:93236886462],
 60: {8702103425: tensor([1, 

In [70]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [71]:
x

(Wrapper)>[PointerTensor | me:1130916636 -> bob:8056263363]

In [72]:
bob._objects

{8702103425: tensor([1, 2, 3, 4, 5]),
 93236886462: tensor([1, 2, 3, 4, 5]),
 17917990552: tensor([1, 2, 3, 4, 5]),
 8056263363: tensor([1, 2, 3, 4, 5])}

In [73]:
x = "asdf"

In [74]:
bob._objects

{8702103425: tensor([1, 2, 3, 4, 5]),
 93236886462: tensor([1, 2, 3, 4, 5]),
 17917990552: tensor([1, 2, 3, 4, 5]),
 8056263363: tensor([1, 2, 3, 4, 5])}

In [75]:
del x

In [76]:
bob._objects

{8702103425: tensor([1, 2, 3, 4, 5]),
 93236886462: tensor([1, 2, 3, 4, 5]),
 17917990552: tensor([1, 2, 3, 4, 5]),
 8056263363: tensor([1, 2, 3, 4, 5])}

In [77]:
bob = bob.clear_objects()
bob._objects

{}

In [78]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [79]:
bob._objects

{31255262088: tensor([1, 2, 3, 4, 5])}

In [80]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [81]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:26243474745 -> bob:2380568225]
Tensor B: tensor([1, 1, 1, 1, 1])

In [82]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [83]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [7]:
from torch import nn, optim

In [8]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [9]:
# A Toy Model
model = nn.Linear(2,1)

In [10]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [11]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(3.5579)
tensor(0.4108)
tensor(0.1248)
tensor(0.0714)
tensor(0.0465)
tensor(0.0309)
tensor(0.0207)
tensor(0.0140)
tensor(0.0095)
tensor(0.0065)
tensor(0.0045)
tensor(0.0031)
tensor(0.0022)
tensor(0.0015)
tensor(0.0011)
tensor(0.0008)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)


In [15]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [16]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [17]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [18]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [19]:
train()

tensor(2.0072, requires_grad=True)
tensor(0.0870, requires_grad=True)
tensor(0.1088, requires_grad=True)
tensor(0.0718, requires_grad=True)
tensor(0.0522, requires_grad=True)
tensor(0.0434, requires_grad=True)
tensor(0.0317, requires_grad=True)
tensor(0.0260, requires_grad=True)
tensor(0.0198, requires_grad=True)
tensor(0.0158, requires_grad=True)
tensor(0.0125, requires_grad=True)
tensor(0.0098, requires_grad=True)
tensor(0.0080, requires_grad=True)
tensor(0.0062, requires_grad=True)
tensor(0.0052, requires_grad=True)
tensor(0.0040, requires_grad=True)
tensor(0.0034, requires_grad=True)
tensor(0.0026, requires_grad=True)
tensor(0.0023, requires_grad=True)
tensor(0.0018, requires_grad=True)
tensor(0.0016, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0009, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0004, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [20]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [21]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [22]:
x = x.send(alice)

In [23]:
bob._objects

{57313588260: tensor([1, 2, 3, 4, 5])}

In [24]:
alice._objects

{68829128709: (Wrapper)>[PointerTensor | alice:68829128709 -> bob:57313588260]}

In [25]:
x

(Wrapper)>[PointerTensor | me:26610996170 -> alice:68829128709]

In [26]:
y = x + x

In [27]:
y

(Wrapper)>[PointerTensor | me:46828037846 -> alice:94783319596]

In [28]:
bob._objects

{57313588260: tensor([1, 2, 3, 4, 5]),
 55264240925: tensor([ 2,  4,  6,  8, 10])}

In [29]:
alice._objects

{68829128709: (Wrapper)>[PointerTensor | alice:68829128709 -> bob:57313588260],
 94783319596: (Wrapper)>[PointerTensor | alice:94783319596 -> bob:55264240925]}

In [30]:
jon = sy.VirtualWorker(hook, id="jon")

In [31]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [32]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:alice #objects:3> while the other is on <VirtualWorker id:jon #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

In [33]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [34]:
bob._objects

{95968246881: tensor([1, 2, 3, 4, 5])}

In [35]:
alice._objects

{64653057476: (Wrapper)>[PointerTensor | alice:64653057476 -> bob:95968246881]}

In [36]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:64653057476 -> bob:95968246881]

In [37]:
bob._objects

{95968246881: tensor([1, 2, 3, 4, 5])}

In [38]:
alice._objects

{}

In [39]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [40]:
bob._objects

{}

In [41]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [42]:
bob._objects

{49208746429: tensor([1, 2, 3, 4, 5])}

In [43]:
alice._objects

{43256825045: (Wrapper)>[PointerTensor | alice:43256825045 -> bob:49208746429]}

In [44]:
del x

In [45]:
bob._objects

{}

In [46]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [47]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [48]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [49]:
bob._objects

{91939424461: tensor([1, 2, 3, 4, 5])}

In [50]:
alice._objects

{10087006416: (Wrapper)>[PointerTensor | alice:10087006416 -> bob:91939424461]}

In [51]:
x.remote_get()

(Wrapper)>[PointerTensor | me:61908968174 -> alice:10087006416]

In [53]:
bob._objects

{}

In [54]:
alice._objects

{10087006416: tensor([1, 2, 3, 4, 5])}

In [55]:
x.move(bob)

(Wrapper)>[PointerTensor | me:61908968174 -> bob:61908968174]

In [56]:
x

(Wrapper)>[PointerTensor | me:61908968174 -> bob:61908968174]

In [57]:
bob._objects

{61908968174: tensor([1, 2, 3, 4, 5])}

In [58]:
alice._objects

{}

In [59]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [60]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [61]:
bob._objects

{12986216181: tensor([1, 2, 3, 4, 5])}

In [62]:
alice._objects

{}

In [63]:
x.move(alice)

(Wrapper)>[PointerTensor | me:57970252523 -> alice:57970252523]

In [64]:
bob._objects

{}

In [65]:
alice._objects

{57970252523: tensor([1, 2, 3, 4, 5])}

In [66]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [67]:
bob._objects

{69332774487: tensor([1, 2, 3, 4, 5])}

In [68]:
alice._objects

{57970252523: tensor([1, 2, 3, 4, 5]),
 26578949750: (Wrapper)>[PointerTensor | alice:26578949750 -> bob:69332774487]}

In [69]:
x.remote_get()

(Wrapper)>[PointerTensor | me:18440741043 -> alice:26578949750]

In [70]:
bob._objects

{}

In [71]:
alice._objects

{57970252523: tensor([1, 2, 3, 4, 5]), 26578949750: tensor([1, 2, 3, 4, 5])}

In [72]:
x.move(bob)

(Wrapper)>[PointerTensor | me:18440741043 -> bob:18440741043]

In [73]:
x

(Wrapper)>[PointerTensor | me:18440741043 -> bob:18440741043]

In [74]:
bob._objects

{18440741043: tensor([1, 2, 3, 4, 5])}

In [75]:
alice._objects

{57970252523: tensor([1, 2, 3, 4, 5])}